### Vlance Crawler

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import re

In [3]:
def open_webdriver():
    driver = webdriver.Chrome()
    driver.get("https://www.vlance.vn/#")

    login_button = driver.find_element(By.ID, "btn-login")
    login_button.click()
    time.sleep(5)

    username = driver.find_element(By.ID, "login_username_header")
    password = driver.find_element(By.ID, "login_password_header")
    username.send_keys("qn9965mh@gmail.com")
    password.send_keys("123456asd*")

    print('Solve captcha manually...............')
    time.sleep(30)

    sumbit_button = driver.find_element(By.ID, "btn-submit-login-header")
    sumbit_button.click()
    
    return driver

In [7]:
def init_dataframe():
    columns = [
        "Type", "Title", "Services", "Skills", "Description", 
        "In4_project", "In4_employment", "Num_applicants", 
        "Applicants", "Duration", "Price", "Link"
    ]

    df = pd.DataFrame(columns=columns)
    return df

In [4]:
#703-936
#crawl page in [page_min,page_max)
def crawl_page(page_min,page_max,driver):
    df=init_dataframe()
    page=page_min
    while True:
        driver.get(f"https://www.vlance.vn/viec-lam-freelance/cpath_cac-cong-viec-it-va-lap-trinh_page_{page}")
        print(f"Page: {page}")
        try:
            WebDriverWait(driver, 23).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "fr-name"))
            )
        except Exception as e:
            print("Error: ", e)
            driver.quit()
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        h3_tags = soup.find_all('h3', attrs={'class':"fr-name block-title"})
        print(len(h3_tags))
        links = []
        Type = []
        Title = []
        Services = []
        Skills = []
        Description = []
        In4_project = []
        In4_employment = []
        Num_applicants = []
        Applicants = []
        Duration = []
        Price = []

        for h3 in h3_tags:
            links.append("https://www.vlance.vn" + h3.find('a')['href'])
            print(h3.find('a')['href'])
            url = f"https://www.vlance.vn{h3.find('a')['href']}"
        
            driver.get(url)
            try:
                WebDriverWait(driver, 30).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h1.title.block-title"))
                )
            except Exception as e:
                print("Error: ", e)
                driver.quit()

            
            time.sleep(10)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Hanlde case secrect project
            secret = soup.find('div', attrs={'class':"span12 btn-connect"})
            if secret:
                print("Secret project")
                Title.append(np.nan)
                Services.append(np.nan)
                Skills.append(np.nan)
                Description.append(np.nan)
                In4_project.append(np.nan)
                In4_employment.append(np.nan)
                Num_applicants.append(np.nan)
                Price.append(np.nan)
                Applicants.append(np.nan)
                Duration.append(np.nan)
                Type.append("Secret project")
                continue

            type = soup.find('span', attrs={'class':"job-progress-title"})
            type_acp_bid = soup.find('div', attrs={'class':"progress-job"})
        

            if type and type.text == "Chiến dịch hiển thị":
                Type.append("Chiến dịch hiển thị")

                title = soup.find('h1', attrs={'class':"title block-title"}).text
                Title.append(title)

                # service = soup.find('div', attrs={'class':"service-title"}).text
                service = np.nan
                Services.append(service)

                description = soup.find('div', attrs={'class':"row-fluid body-view review-text"}).text
                Description.append(description)

                # skills = soup.find('div', attrs={'class':"span10 skill"}).text
                skills = np.nan
                Skills.append(skills)

                in4_project = soup.find('div', attrs={'class':"description-job"})
                if in4_project:
                    In4_project.append(in4_project.text)
                else:
                    In4_project.append(np.nan)

                in4_employment = soup.find('div', attrs={'class':"info-employment"})
                if in4_employment:
                    In4_employment.append(in4_employment.text)
                else:
                    In4_employment.append(np.nan)


                num_applicants = soup.find('span', attrs={'class':"bid-samples"}).text
                Num_applicants.append(num_applicants)

                budget = soup.find('div', attrs={'class':"span4 client-bidding-between"}).text
                Price.append(budget)

                duration_avg = soup.find('span', attrs={'class':"duration-average"}).text
                Duration.append(duration_avg)

                applicants = np.nan
                Applicants.append(applicants)
                
            else:
                if type and type.text == "Đăng việc":
                    Type.append("Đăng việc")
                    
                    title = soup.find('h1', attrs={'class':"title block-title"}).text
                    Title.append(title)

                    service = soup.find('div', attrs={'class':"service-title"})
                    if service:
                        Services.append(service.text)
                    else: 
                        Services.append(np.nan)

                    description = soup.find('div', attrs={'class':"span10 description"}).text
                    Description.append(description)

                    skills = soup.find('div', attrs={'class':"span10 skill"})
                    if skills:
                        Skills.append(skills.text)
                    else:
                        Skills.append(np.nan)

                    in4_project = soup.find('div', attrs={'class':"description-job"})
                    if in4_project:
                        In4_project.append(in4_project.text)
                    else:
                        In4_project.append(np.nan)

                    in4_employment = soup.find('div', attrs={'class':"info-employment"})
                    if in4_employment:
                        In4_employment.append(in4_employment.text)
                    else:
                        In4_employment.append(np.nan)

                    bid_counter = soup.find('span', attrs={'class':"bid-counter"}).text
                    Num_applicants.append(bid_counter)

                    bid_price = soup.find('div', attrs={'class':"span7 offset1 client-bidding-between-new"}).text
                    Price.append(bid_price)

                    duration_avg = soup.find('span', attrs={'class':"duration-average"}).text
                    Duration.append(duration_avg)

                    applicants = soup.find('div', attrs={'class':"row-fluid container list-bid-new"})
                    list_applicants = []
                    if applicants:
                        a_tags = applicants.find_all('h3', attrs={'class':"title"})
                        for user_href in a_tags:
                            list_applicants.append(user_href.find('a')['href'])
                    Applicants.append(list_applicants)

                else:
                    if type_acp_bid and re.search(r"Nhận chào giá", type_acp_bid.text.strip()):
                        print("fuck off")
                        Type.append("Nhận chào giá")
                        title = soup.find('h1', attrs={'class':"title block-title"}).text
                        Title.append(title)
                        #print(title)

                        service = soup.find('div', attrs={'class':"service-title"})
                        if service:
                            Services.append(service.text)
                        else: 
                            Services.append(np.nan)

                        description = soup.find('div', attrs={'class':"span10 description"}).text
                        Description.append(description)

                        skills = soup.find('div', attrs={'class':"span10 skill"})
                        if skills:
                            Skills.append(skills.text)
                        else:
                            Skills.append(np.nan)

                        in4_project = soup.find('div', attrs={'class':"description-job"})
                        if in4_project:
                            In4_project.append(in4_project.text)
                        else:
                            In4_project.append(np.nan)

                        in4_employment = soup.find('div', attrs={'class':"info-employment"})
                        if in4_employment:
                            In4_employment.append(in4_employment.text)
                        else:
                            In4_employment.append(np.nan)

                        bid_counter = soup.find('span', attrs={'class':"bid-counter"}).text
                        Num_applicants.append(bid_counter)

                        bid_price = soup.find('div', attrs={'class':"span6 client-bidding-between"}).text
                        Price.append(bid_price)

                        duration_avg = soup.find('span', attrs={'class':"duration-average"}).text
                        Duration.append(duration_avg)

                        applicants = soup.find_all('div', attrs={'class':"profile-job-left-bottom"})
                        list_applicants = []
                        if applicants:
                            for user_href in applicants:
                                list_applicants.append(user_href.find('a')['href'])
                        Applicants.append(list_applicants)


                    else:
                        print("So sick and tired")
                        Title.append(np.nan)
                        Services.append(np.nan)
                        Skills.append(np.nan)
                        Description.append(np.nan)
                        In4_project.append(np.nan)
                        In4_employment.append(np.nan)
                        Num_applicants.append(np.nan)
                        Price.append(np.nan)
                        Applicants.append(np.nan)
                        Duration.append(np.nan)
                        Type.append(np.nan)
        
        one_page = pd.DataFrame({
                "Type": Type,
                "Title": Title,
                "Services": Services,
                "Skills": Skills,
                "Description": Description,
                "In4_project": In4_project,
                "In4_employment": In4_employment,
                "Num_applicants": Num_applicants,
                "Applicants": Applicants,
                "Duration": Duration,
                "Price": Price,
                "Link": links
            })

        df = pd.concat([df, one_page], ignore_index=True)

        page += 1
        if page == page_max:
            df.to_csv(f"job_posts_{page_min}_{page_max}.csv", index=False)
            break
    


In [19]:
list_driver=[]
#batch_list=[[733,763],[763,793],[793,823],[823,853],[853,883],[883,913],[913,937]]
batch_list=[[913,937]]
args_list=[]
for i in range(len(batch_list)):
    driver=open_webdriver()
    list_driver.append(driver)
    args_list.append((batch_list[i][0],batch_list[i][1],driver))


Solve captcha manually...............


In [11]:
# crawl_page(793,813,list_driver[0])

In [20]:
import threading

threads = []
for args in args_list:
    threads.append(threading.Thread(target=crawl_page, args=args))
    
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()



Page: 913
10
/viec-freelance/minh-can-mot-ban-chuyen-lam-tool-voi-selenium
/viec-freelance/lam-website-affiliate-lay-du-lieu-tu-shopee-lazada-tiki
fuck off
/viec-freelance/thiet-ke-theme-web-e-commerce
/viec-freelance/minh-can-nguoi-lam-tool-check-acc-gamvip
/viec-freelance/xay-dung-middle-ware-he-thong-cham-soc-khach-hang-qua-zalo-viber
/viec-freelance/xay-dung-web-tong-hop-bao-cao-du-lieu-dung-power-bi
/viec-freelance/web-va-app-ban-hoa-tuoi
fuck off
/viec-freelance/app-quan-ly-toa-nha
fuck off
/viec-freelance/cai-dat-vps-de-hosting-website-wordpress
/viec-freelance/quan-ly-co-dong-online
Page: 914
10
/viec-freelance/viet-app-python-ket-noi-facebook-va-zalo-de-doc-tin-nhan
/viec-freelance/lam-mot-mobile-app-de-giao-viec
/viec-freelance/install-va-action-tron-goi-source-app-tu-codecanyon
Secret project
/viec-freelance/admin-cho-webstie-nganh-hang-thoi-trang
/viec-freelance/lam-1-ung-dung-android-don-gian-fake-gps
/viec-freelance/thiet-ke-trang-web-huong-dan-tai-mobile-app-giai-tri
/vi

In [11]:
print("Length of Type:", len(Type))
print("Length of Title:", len(Title))
print("Length of Services:", len(Services))
print("Length of Skills:", len(Skills))
print("Length of Description:", len(Description))
print("Length of In4_project:", len(In4_project))
print("Length of In4_employment:", len(In4_employment))
print("Length of Num_applicants:", len(Num_applicants))
print("Length of Applicants:", len(Applicants))
print("Length of Duration:", len(Duration))
print("Length of Price:", len(Price))
Type

Length of Type: 10
Length of Title: 10
Length of Services: 10
Length of Skills: 10
Length of Description: 10
Length of In4_project: 10
Length of In4_employment: 10
Length of Num_applicants: 10
Length of Applicants: 10
Length of Duration: 10
Length of Price: 10


['Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Nhận chào giá',
 nan,
 'Đăng việc']

In [13]:
df.head(10)

,Type,Title,Services,Skills,Description,In4_project,In4_employment,Num_applicants,Applicants,Duration,Price,Link
0,Nhận chào giá,\nThiết kế tool tự động điền captcha khi đăng ...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,NaN,"\n\nChào các bạn, \nHiện tại mình đang cần thi...",\n\nID dự án\n37869\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ph...,\n Chào giá: ...,[/freelancer/tuan-thanh-19],\n Trung bình: 3...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/thiet-ke-...
1,Nhận chào giá,\nThiết kế web bán khoá theo mẫu có sẵn Hết hạ...,\nDịch vụ cần thuê:\n\n\nDựng website bán hàng...,"\nKỹ năng\n\nSEO , \nNhập dữ liệu , \nThiết kế...",\n\nThiết kế web bán khoá điện tử \nCần bạn nà...,\n\nID dự án\n37865\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Na...,\n Chào giá: ...,"[/freelancer/thanh-tien, /freelancer/nghiem-qu...",\n Trung bình: 7...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/thiet-ke-...
2,Nhận chào giá,\nSet up proxy 3g Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,"\nKỹ năng\n\nLinux , \nAndroid , \nPython",\n\nTạo hệ thống tự động tạo proxy từ usb 3g/4...,\n\nID dự án\n37864\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Wa...,\n Chào giá: ...,[/freelancer/tran-5],\n Trung bình: 3...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/set-up-pr...
3,Nhận chào giá,\nDesigner cho project vuejs Hết hạn nhận chào...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,"\nKỹ năng\n\nBootstrap , \nresponsive",\n\n- Chỉnh lại màu của cả website cho hài hòa...,\n\nID dự án\n37861\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Th...,\n Chào giá: ...,"[/freelancer/dottp-tech, /freelancer/pham-van-...",\n Trung bình: 4...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/designer-...
4,Nhận chào giá,\nViết phần mềm và web cho doanh nghiệp Hết hạ...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,NaN,\n\nLàm việc theo giờ hoặc theo dự án đều được...,\n\nID dự án\n37860\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n BA...,\n Chào giá: ...,"[/freelancer/vien-nguyen-9, /freelancer/hoang-...",\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/viet-phan...
5,Nhận chào giá,\nLiệt kê theo điều kiện trên exel dùng VBA Đã...,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,\nKỹ năng\n\nVBA,\n\nMình có một file excel cần xữ lý dữ liệu n...,\n\nID dự án\n37859\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n ph...,\n Chào giá: ...,"[/freelancer/le-thanh-phuoc-1, /freelancer/pho...",\n Trung bình: 4...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/liet-ke-t...
6,Nhận chào giá,\n10 cộng tác viên sử dụng ứng dụng Android tr...,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nAndroid",\n\nĐiều kiện: \nSmart phone chạy Android 6.0 ...,\n\nID dự án\n37855\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Lo...,\n Chào giá: ...,"[/freelancer/hung-doan-minh, /freelancer/thuy-...",\n Trung bình: 2...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/10-cong-t...
7,Nhận chào giá,\nNâng cấp giao diện trang web joomla Hết hạn ...,\nDịch vụ cần thuê:\n\n\nThiết kế giao diện we...,"\nKỹ năng\n\nJoomla , \nPHP",\n\n-Chỉnh sửa lại giao diện trang home : www....,\n\nID dự án\n37854\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,"[/freelancer/nguyen-trung-23, /freelancer/nam-...",\n Trung bình: 6...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/nang-cap-...
8,Nhận chào giá,\nThiết kế app tính toán lô đề theo công thức ...,\nDịch vụ cần thuê:\n\n\nLàm Mobile App theo y...,"\nKỹ năng\n\nTrình bày , \nLập trình ứng dụng",\n\nỨng dụng di động cho phép tính toán lô đề ...,\n\nID dự án\n37852\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Xu...,\n Chào giá: ...,[],\n Trung bình: 0...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/thiet-ke-...
9,Nhận chào giá,\nĐánh dấu dòng và cột của ô trong excel khi d...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,"\nKỹ năng\n\nVBA , \nMS Excel",\n\nYêu cầu : \n- Tạo Addin đánh dấu dòng và c...,\n\n

In [14]:
df.to_csv("job_posts.csv", index=False)

## Từ khúc sau này là code test crawl freelancer

In [16]:
url = "https://www.vlance.vn/viec-freelance/long-hinh-anh-quy-hoach-vao-google-map"


In [28]:
print(title)
print(service)
print(skills)
print(description)
print(in4_project)
print(in4_employment)
print(bid_counter)
print(bid_price)
print(duration_avg)
list_applicants


Lồng hình ảnh quy hoạch vào google map Hết hạn nhận chào giá


Dịch vụ cần thuê:


Phát triển ứng dụng web 



Kỹ năng

Bất động sản  


Tôi có 1 hình ảnh của 1 dự án bất động sản đang triển khai, bao gồm các lô đất được phân trên hình ảnh đó, hãy giúp tôi lồng nó vào google map để người dùng dễ hình dung được là lô nào ở đâu trên google map.




ID dự án
79847
Ngày đăng

                                            07/10/2024, 17:11                                    
Chỉ còn
Hết hạn nhận chào giá
Địa điểm

                Hà Nội            
Ngân sách

                5.000.000đ                                                             - 20.000.000đ                                                
Hình thức làm việc

                                                                Làm online                                                        
Hình thức trả lương

                                    Trả theo dự án                            












                        Thuy

['/freelancer/nguyen-nhat-hao-1',
 '/freelancer/cong-ty-phan-mem-nextcore',
 '/freelancer/tuan-anh-truong-10',
 '/freelancer/pm-hung']

In [3]:
url = "https://www.vlance.vn/freelancer/pm-hung"
driver.get(url)

In [4]:
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.id_profile"))
    )
except Exception as e:
    print("Error: ", e)
    driver.quit()

In [5]:
try:
    see_more_skill = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "see-more-skill"))
    )
    see_more_skill.click()
    print("Clicked on 'see-more-skill'")
except Exception as e:
    print("Element 'see-more-skill' not found or not clickable:", e)

Clicked on 'see-more-skill'


In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
id = soup.find('div', attrs={'class':"id_profile"}).text
name = soup.find('span', attrs={'itemprop':"name"}).text
title = soup.find('span', attrs={'itemprop':"title editable tf300"}).text
region = soup.find('div', attrs={'class':"regional-price"}).text
overview = soup.find('div', attrs={'class':"overview overview-mobile"}).text
services = soup.find('div', attrs={'class':"service-freelancer"}).text
summary = soup.find('div', attrs={'class':"summary-profile profile-rate summary-title summary-block"}).text
summary_working = soup.find('div', attrs={'class':"summary-profile profile-rate summary-working"}).text


In [16]:
# print(id)
# print(name)
# print(title)
# print(region)
#print(overview)
#print(services)
#summary
print(summary_working)


Làm việc


Chuyên gia (trên 5 năm kinh nghiệm) 


